Je vais ici essayer de créer des bases de données avec tous les url des articles suceptibles de nous intéresser parmi tous les éléments que je n'ai pas récupéré manuellement

In [11]:
!pip install bs4
!pip install requests
!pip install lxml
!pip install selenium
!pip install --upgrade selenium

import requests
import lxml
from lxml import html
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import pandas as pd
import numpy as np 
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

# SOS Education #

In [3]:
xpath = '//*[(@id = "articles")]//*[contains(concat( " ", @class, " " ), concat( " ", "vc_col-sm-12", " " ))]'
url = "https://soseducation.org/notes-etudes-entretiens-tribunes/la-question-transgenre-chez-les-enfants/"

In [14]:
# Fonction pour récupérer le contenu HTML d'une URL
def get_html_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Erreur {response.status_code} lors de la récupération de la page {url}")
        return None

# Récupérer le contenu HTML de la page
html_content = get_html_content(url)

# Vérifier si le contenu a été récupéré avec succès
if html_content:
    # Utiliser BeautifulSoup pour traiter le HTML
    soup = BeautifulSoup(html_content, 'lxml')

    # Utiliser find_all avec find pour récupérer les éléments correspondants à l'expression XPath
    elements = soup.find_all("a")
    
    # Extraire les liens des éléments trouvés
    links = [element.a['href'] for element in elements if element.a]
    
links

[]

In [9]:
# Supposons que votre contenu HTML est dans la variable html_content
soup = BeautifulSoup(html_content, 'html.parser')

# Trouver tous les liens avec la classe "vc_general"
links = soup.find_all('a', class_='vc_general')

In [13]:
response = requests.get(url)
tree = html.fromstring(response.content)

# Utilisez XPath pour trouver les liens dans la partie "lettres d'information"
links = tree.xpath(xpath)
links

[<Element div at 0x7f2af1362cf0>, <Element div at 0x7f2af155b880>]

Ok j'ai pas réussi à scrapper ce site

# TRADFEM #

In [36]:
url = 'https://tradfem.wordpress.com/'
# Spécifiez le chemin local du chromedriver
chromedriver_path = "./chromedriver.exe"

# Configurez les options du navigateur si nécessaire
chrome_options = webdriver.ChromeOptions()
# Par exemple, pour exécuter en mode têteless (sans interface graphique), décommentez la ligne suivante :
# chrome_options.add_argument('--headless')

# Ajoutez le chemin du pilote aux options
chrome_options.add_argument(f"webdriver.chrome.driver={chromedriver_path}")

# Initialisez le pilote Chrome avec les options
driver = webdriver.Chrome(options=chrome_options)

# Utilisez le pilote pour accéder aux pages Web, effectuer des actions, etc.
driver.get(url)

# Défilement vers le bas pour charger plus de contenu
for _ in range(3):  # Changez le nombre d'itérations selon vos besoins
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(1)  # Pause pour laisser le temps au contenu de se charger

# Attendez quelques secondes supplémentaires pour vous assurer que tout le contenu est chargé
time.sleep(2)

# Obtenez le contenu de la page après le défilement
page_content = driver.page_source

# Fermez le navigateur automatisé
driver.quit()

# Utilisez BeautifulSoup pour analyser le contenu
soup = BeautifulSoup(page_content, "html.parser")
liens = soup.find_all("a")
liens

WebDriverException: Message: Service /home/onyxia/.cache/selenium/chromedriver/linux64/119.0.6045.105/chromedriver unexpectedly exited. Status code was: 127


Grosses difficultés à gérer Seleium et le WebDriver

# Christined75 #

In [3]:
url = "https://christineld75.wordpress.com/sommaire-des-articles/"
page = requests.get(url)

# Ajoutez une pause de 1 seconde
time.sleep(1)

soup = BeautifulSoup(page.content, "html.parser")
html_liens = soup.find_all("a")
liens = [link.get("href") for link in html_liens if link.get("href")]

In [49]:
#Ici j'essaie de récupérer les titres mais c compliqué

url = "https://christineld75.wordpress.com/sommaire-des-articles/"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
list_titre = []

# Récupérer toutes les balises <a> avec la classe "strong" sur la page
balises_a_strong = soup.find_all("a")

# Itérer sur les balises <a> pour extraire les liens et les textes associés
for balise_a in balises_a_strong:
    lien = balise_a.get("href")
    texte_apres_lien = balise_a.find_next_sibling("strong").text.strip() if balise_a.find_next_sibling("strong") else ""
    if texte_apres_lien != "":
        list_titre.append(texte_apres_lien)
# Afficher la liste des titres
list_titre

[',\xa0Féminisme et faux amiEs : récupération, dévoiement\xa0,\xa0Féminisme, politique et société']

In [9]:
contenus = []

for lien in liens[0:100]:
    try:
        # Obtenir l'URL de chaque lien
        url_page_referree = lien

        # Envoyer une requête pour obtenir le contenu HTML de la page référencée
        response_page_referree = requests.get(url_page_referree)
        response_page_referree.raise_for_status()  # Vérifier si la requête a réussi

        # Utiliser BeautifulSoup pour analyser le HTML de la page référencée
        soup_page_referree = BeautifulSoup(response_page_referree.content, 'html.parser')

        # Trouver le contenu spécifique que vous souhaitez extraire (ajustez en fonction de la structure HTML)
        content = soup_page_referree.find('div', {'class': 'post'})
        content_text = content.text.strip() if content else "Contenu non trouvé"

        # Ajouter le contenu à la liste
        contenus.append(content_text)

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de {url_page_referree}:")

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Observatoire de la Petite Sirène #

In [12]:
url = "https://www.observatoirepetitesirene.org/nos-publications"
page = requests.get(url)

# Ajoutez une pause de 1 seconde
time.sleep(1)

soup = BeautifulSoup(page.content, "html.parser")
html_liens = soup.find_all("a")
liens = [link.get("href") for link in html_liens if link.get("href")]


In [14]:

# La chaîne de caractères à rechercher
prefixe = "https://www.observatoirepetitesirene.org/post"

# Filtrer les éléments de la liste commençant par le préfixe
articles_filtres = [url for url in liens if url.startswith(prefixe)]

#On obtient pas les liens voulus. Je corrige plus tard

['https://www.observatoirepetitesirene.org/post/psychological-outcomes-of-12-15-year-olds-with-gender-dysphoria-receiving-pubertal-suppression', 'https://www.observatoirepetitesirene.org/post/have-the-psychiatric-needs-of-people-seeking-gender-reassignment-changed-as-their-numbers-increase', 'https://www.observatoirepetitesirene.org/post/adolescent-development-and-psychosocial-functioning-after-starting-cross-sex-hormones', 'https://www.observatoirepetitesirene.org/post/two-years-of-gender-identity-service-for-minors', 'https://www.observatoirepetitesirene.org/post/detransition-and-desistance-among-previously-trans-identified-young-adults', 'https://www.observatoirepetitesirene.org/post/nouvelle-étude-sur-les-détransitions', 'https://www.observatoirepetitesirene.org/post/gender-detransition-a-critical-review-of-the-literature-1', 'https://www.observatoirepetitesirene.org/post/breastfeeding-grief-after-chest-masculinisation-mastectomy-and-detransition-a-case-report-with-less', 'https://

# Ypomoni #

In [17]:
# URL de la page contenant les liens vers les PDF
url_de_la_page = "https://ypomoni.org/sante/"

def extract_ypo(url):
    liste_url =  []

    # Télécharger le contenu de la page
    response = requests.get(url_de_la_page)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # Utiliser une expression régulière pour trouver les liens vers les fichiers PDF
        liens_pdfs = soup.find_all('a', href=re.compile(r'\.pdf$'))

        # Afficher les URLs des PDF
        for lien_pdf in liens_pdfs:
            url_pdf = lien_pdf['href']
            liste_url.append(url_pdf)
    else:
        print(f"Erreur de requête sur la page {url_de_la_page}")
    return liste_url

extract_ypo(url_de_la_page)

['https://www.academie-medecine.fr/wp-content/uploads/2022/02/22.2.25-Communique-PCRA-19-Medecine-et-transidentite-genre.pdf',
 'https://solidarites-sante.gouv.fr/IMG/pdf/rapport_sante_des_personnes_trans_2022.pdf',
 'https://www.observatoirepetitesirene.org/_files/ugd/49b30a_b7027bdd0464478493dccdfeb47b9d65.pdf']

On va scrapper direct le contenu car finalement, pas grand chose d'exploitable.
Je le fais plus tard car dans l'immédiat, sur ma tablette, c'est pas du tout pratique.